# 11.4.1 데이터 준비

In [1]:
from urllib.request import urlretrieve
from zipfile import ZipFile


urlretrieve('http://doc.mindscale.kr/km/unstructured/dog-vs-cat.zip',
            'dog-vs-cat.zip')

with ZipFile('dog-vs-cat.zip') as z:
    z.extractall()

In [2]:
from keras.preprocessing.image import ImageDataGenerator

train = ImageDataGenerator(rescale=1.0/255).flow_from_directory(
    'dog-vs-cat/train',
    target_size=(100, 100),
    class_mode='binary')

valid = ImageDataGenerator(rescale=1.0/255).flow_from_directory(
    'dog-vs-cat/validation',
    target_size=(100, 100),
    class_mode='binary',
    shuffle=False)

Using TensorFlow backend.


Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


# 11.4.2. 합성곱 신경망과 텐서보드

In [15]:
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.models import Sequential

In [16]:
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model1.add(MaxPooling2D((2, 2)))
model1.add(Flatten())
model1.add(Dense(1, activation='sigmoid'))

In [17]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 76832)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 76833     
Total params: 77,729
Trainable params: 77,729
Non-trainable params: 0
_________________________________________________________________


In [18]:
from keras.optimizers import Adam, RMSprop

In [19]:
model1.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam())

### 텐서보드

In [20]:
from keras.callbacks import EarlyStopping, TensorBoard

In [21]:
history1 = model1.fit_generator(
    train, validation_data=valid, epochs=30,
    callbacks=[
        EarlyStopping(monitor = "val_loss", patience=2),
        TensorBoard(log_dir='log_model1')
    ])

Epoch 1/30
63/63 [==============================] - 21s 339ms/step - loss: 0.8042 - acc: 0.5094 - val_loss: 0.6611 - val_acc: 0.5325
Epoch 2/30
63/63 [==============================] - 20s 324ms/step - loss: 0.6301 - acc: 0.6493 - val_loss: 0.6298 - val_acc: 0.6350
Epoch 3/30
63/63 [==============================] - 21s 327ms/step - loss: 0.5302 - acc: 0.7559 - val_loss: 0.5986 - val_acc: 0.6825
Epoch 4/30
63/63 [==============================] - 22s 347ms/step - loss: 0.4272 - acc: 0.8209 - val_loss: 0.6147 - val_acc: 0.6650
Epoch 5/30
63/63 [==============================] - 19s 309ms/step - loss: 0.3460 - acc: 0.8864 - val_loss: 0.6217 - val_acc: 0.6725


# 11.4.3. 더 깊은 신경망

In [22]:
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Flatten())
model2.add(Dense(1, activation='sigmoid'))

In [23]:
model2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam())

In [24]:
history2 = model2.fit_generator(
    train, validation_data=valid, epochs=30,
    callbacks=[
        EarlyStopping(monitor = "val_loss", patience=2),
        TensorBoard(log_dir='log_model2')
    ])

Epoch 1/30
63/63 [==============================] - 36s 565ms/step - loss: 0.6833 - acc: 0.5506 - val_loss: 0.6892 - val_acc: 0.4975
Epoch 2/30
63/63 [==============================] - 34s 541ms/step - loss: 0.6422 - acc: 0.6176 - val_loss: 0.6440 - val_acc: 0.6288
Epoch 3/30
63/63 [==============================] - 31s 496ms/step - loss: 0.5821 - acc: 0.6969 - val_loss: 0.5976 - val_acc: 0.6775
Epoch 4/30
63/63 [==============================] - 28s 437ms/step - loss: 0.5279 - acc: 0.7421 - val_loss: 0.5840 - val_acc: 0.6975
Epoch 5/30
63/63 [==============================] - 28s 449ms/step - loss: 0.4829 - acc: 0.7703 - val_loss: 0.6000 - val_acc: 0.7025
Epoch 6/30
63/63 [==============================] - 28s 452ms/step - loss: 0.4491 - acc: 0.8011 - val_loss: 0.6059 - val_acc: 0.6613


# 11.4.4. 데이터 증강

In [25]:
img_gen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,      # 40도까지 회전
        width_shift_range=0.2,  # 20%까지 좌우 이동
        height_shift_range=0.2, # 20%까지 상하 이동
        shear_range=0.2,        # 20%까지 기울임
        zoom_range=0.2,         # 20%까지 확대
        horizontal_flip=True,   # 좌우 뒤집기
    )

train_ag = img_gen.flow_from_directory(
        'dog-vs-cat/train',
        target_size=(100, 100),
        class_mode='binary')

Found 2000 images belonging to 2 classes.


In [26]:
model3 = Sequential()
model3.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model3.add(MaxPooling2D((2, 2)))
model3.add(Conv2D(32, (3, 3), activation='relu'))
model3.add(MaxPooling2D((2, 2)))
model3.add(Flatten())
model3.add(Dense(1, activation='sigmoid'))

In [27]:
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 16928)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 16929     
Total params: 27,073
Trainable params: 27,073
Non-trainable params: 0
_________________________________________________________________


In [28]:
model3.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam())

In [29]:
history3 = model3.fit_generator(
    train, validation_data=valid, epochs=30,
    callbacks=[
        EarlyStopping(monitor = "val_loss", patience=2),
        TensorBoard(log_dir='log_model3')
    ])

Epoch 1/30
63/63 [==============================] - 30s 476ms/step - loss: 0.6936 - acc: 0.5496 - val_loss: 0.6797 - val_acc: 0.5537
Epoch 2/30
63/63 [==============================] - 29s 467ms/step - loss: 0.6505 - acc: 0.6220 - val_loss: 0.6323 - val_acc: 0.6813
Epoch 3/30
63/63 [==============================] - 28s 444ms/step - loss: 0.6098 - acc: 0.6786 - val_loss: 0.6139 - val_acc: 0.6863
Epoch 4/30
63/63 [==============================] - 31s 499ms/step - loss: 0.5591 - acc: 0.7242 - val_loss: 0.6000 - val_acc: 0.6925
Epoch 5/30
63/63 [==============================] - 34s 537ms/step - loss: 0.4936 - acc: 0.7584 - val_loss: 0.5977 - val_acc: 0.6975
Epoch 6/30
63/63 [==============================] - 33s 523ms/step - loss: 0.4555 - acc: 0.7872 - val_loss: 0.5870 - val_acc: 0.6975
Epoch 7/30
63/63 [==============================] - 28s 442ms/step - loss: 0.3964 - acc: 0.8249 - val_loss: 0.5852 - val_acc: 0.7113
Epoch 8/30
63/63 [==============================] - 28s 446ms/step - 

# 11.4.5. 드롭아웃과 학습률 조정 그리고 ModelCheckpoint

In [30]:
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint

In [31]:
model4 = Sequential()
model4.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model4.add(MaxPooling2D((2, 2)))
model4.add(Conv2D(64, (3, 3), activation='relu'))
model4.add(MaxPooling2D((2, 2)))
model4.add(Conv2D(128, (3, 3), activation='relu'))
model4.add(MaxPooling2D((2, 2)))
model4.add(Flatten())
model4.add(Dropout(0.5))
model4.add(Dense(512, activation='relu'))
model4.add(Dense(1, activation='sigmoid'))

In [32]:
model4.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=RMSprop(lr=0.0001))

In [33]:
history4 = model4.fit_generator(
    train, validation_data=valid, epochs=30,
    callbacks=[
        ModelCheckpoint('model4-{epoch:02d}.hdf5', save_best_only=True),
        TensorBoard(log_dir='log_model4')
    ])

Epoch 1/30
63/63 [==============================] - 58s 917ms/step - loss: 0.6954 - acc: 0.5352 - val_loss: 0.6880 - val_acc: 0.5075
Epoch 2/30
63/63 [==============================] - 62s 986ms/step - loss: 0.6690 - acc: 0.6032 - val_loss: 0.6771 - val_acc: 0.5525
Epoch 3/30
63/63 [==============================] - 57s 904ms/step - loss: 0.6351 - acc: 0.6419 - val_loss: 0.6379 - val_acc: 0.6338
Epoch 4/30
63/63 [==============================] - 55s 876ms/step - loss: 0.6038 - acc: 0.6741 - val_loss: 0.6357 - val_acc: 0.6562
Epoch 5/30
63/63 [==============================] - 56s 881ms/step - loss: 0.5813 - acc: 0.6925 - val_loss: 0.6127 - val_acc: 0.6763
Epoch 6/30
63/63 [==============================] - 55s 879ms/step - loss: 0.5411 - acc: 0.7332 - val_loss: 0.6204 - val_acc: 0.6613
Epoch 7/30
63/63 [==============================] - 55s 868ms/step - loss: 0.5257 - acc: 0.7306 - val_loss: 0.6121 - val_acc: 0.6763
Epoch 8/30
63/63 [==============================] - 54s 861ms/step - 

### 저장된 모델 목록 보기

In [34]:
import glob

In [35]:
model_files = glob.glob('model4-*.hdf5')

In [36]:
model_files

['model4-02.hdf5',
 'model4-03.hdf5',
 'model4-12.hdf5',
 'model4-04.hdf5',
 'model4-08.hdf5',
 'model4-09.hdf5',
 'model4-05.hdf5',
 'model4-07.hdf5',
 'model4-01.hdf5']

In [37]:
last_model = sorted(model_files)[-1]

In [38]:
last_model

'model4-12.hdf5'

### Colab에서 다운로드 받기

In [39]:
from google.colab import files

files.download(last_model)


ImportError: No module named 'google.colab'

### 모델 다시 불러오기

In [ ]:
import keras.models

In [ ]:
model5 = keras.models.load_model(last_model)